# Generate Melody Script 256 LSTM




In [ ]:
# Install necessary packages
!pip install pretty_midi tensorflow

# Import essential libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pretty_midi
import random

# Suppress TensorFlow warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

# 1. Function to Load the Trained LSTM Model and Pitch Mappings

def load_trained_model(model_path='final_lstm_music_model.keras',
                       pitch_to_idx_path='pitch_to_index_lstm.pkl',
                       idx_to_pitch_path='index_to_pitch_lstm.pkl'):
    """
    Loads the pre-trained LSTM model along with pitch mappings.

    Args:
        model_path (str): Path to the saved LSTM model.
        pitch_to_idx_path (str): Path to the pitch-to-index mapping pickle file.
        idx_to_pitch_path (str): Path to the index-to-pitch mapping pickle file.

    Returns:
        tf.keras.Model: Loaded LSTM model.
        dict: Mapping from pitches to indices.
        dict: Mapping from indices to pitches.
    """
    print("Loading the pre-trained LSTM model...")
    try:
        # Load the model
        lstm_model = load_model(model_path, compile=False)
        print("LSTM model successfully loaded.\n")
    except Exception as e:
        print(f"Error loading the model: {e}")
        raise e

    print("Retrieving pitch mappings...")
    try:
        with open(pitch_to_idx_path, 'rb') as pt_file:
            pitch_to_idx = pickle.load(pt_file)
        with open(idx_to_pitch_path, 'rb') as itp_file:
            idx_to_pitch = pickle.load(itp_file)
        print("Pitch mappings successfully retrieved.\n")
    except Exception as e:
        print(f"Error loading pitch mappings: {e}")
        raise e

    return lstm_model, pitch_to_idx, idx_to_pitch

# 2. Function to Load Seed Sequences for Initialization

def load_seed_sequences(seeds_file='lstm_train_seeds.pkl'):
    """
    Loads serialized seed sequences to initialize the music generation.

    Args:
        seeds_file (str): Path to the serialized training sequences pickle file.

    Returns:
        list: List of seed sequences with encoded pitch indices.
    """
    print("Loading seed sequences for generation...")
    try:
        with open(seeds_file, 'rb') as f:
            seed_sequences = pickle.load(f)
        print(f"Successfully loaded {len(seed_sequences)} seed sequences.\n")
    except Exception as e:
        print(f"Error loading seed sequences: {e}")
        raise e
    return seed_sequences

# 3. Function to Generate New Music Sequences Using the LSTM Model

def generate_music_sequence(model, seeds, idx_to_pitch, seq_length=64,
                            generation_steps=500, temperature=1.0, activation_threshold=0.5):
    """
    Generates a new sequence of musical pitches using the trained LSTM model.

    Args:
        model (tf.keras.Model): Trained LSTM Keras model.
        seeds (list): List of seed sequences for initialization.
        idx_to_pitch (dict): Dictionary mapping indices to pitch values.
        seq_length (int): Number of previous time steps to consider for prediction.
        generation_steps (int): Total number of new time steps to generate.
        temperature (float): Controls the randomness of predictions.
        activation_threshold (float): Threshold to determine active pitches.

    Returns:
        list: Generated sequence of pitch indices over the specified steps.
    """
    # Select a random seed from the training sequences
    seed_sequence = random.choice(seeds)
    print("Selected an initial seed sequence for generation.\n")

    # Initialize the sequence with the seed
    generated_sequence = seed_sequence.copy()

    print("Commencing the music generation process...")
    for step in range(generation_steps):
        # Prepare the input array by encoding the last 'seq_length' time steps
        input_array = np.zeros((1, seq_length, len(idx_to_pitch)), dtype=np.float32)
        for t in range(seq_length):
            if len(generated_sequence) >= seq_length:
                current_step = generated_sequence[-seq_length + t]
            else:
                current_step = generated_sequence[:t+1]
            for pitch_idx in current_step:
                if 0 <= pitch_idx < len(idx_to_pitch):
                    input_array[0, t, pitch_idx] = 1.0  # Multi-hot encoding

        # Predict the next set of pitches
        predictions = model.predict(input_array, verbose=0)[0, -1]  # Shape: (num_pitches,)

        # Apply temperature scaling to introduce randomness
        if temperature <= 0:
            temperature = 1.0  # Prevent division by zero or negative temperature
        scaled_predictions = np.log(predictions + 1e-8) / temperature
        exp_predictions = np.exp(scaled_predictions)
        probabilities = exp_predictions / np.sum(exp_predictions)

        # Determine which pitches are active based on the threshold
        active_pitches = [idx for idx, prob in enumerate(probabilities) if prob > activation_threshold]

        if not active_pitches:
            # Ensure at least one pitch is active by selecting the highest probability pitch
            active_pitches = [np.argmax(probabilities)]

        # Append the new pitches to the generated sequence
        generated_sequence.append(active_pitches)

        # Optional: Display progress at intervals
        if (step + 1) % 100 == 0:
            print(f"Progress Update: Generated {step + 1} steps...")

    print(f"\nMusic generation completed: {generation_steps} new steps generated.\n")
    return generated_sequence

# 4. Function to Convert Generated Sequences to a MIDI File

def convert_sequence_to_midi(generated_sequence, idx_to_pitch, output_file='lstm_generated_music.mid',
                             step_duration=0.25, velocity=100):
    """
    Converts a generated sequence of pitch indices into a MIDI file.

    Args:
        generated_sequence (list): List of generated pitch indices.
        idx_to_pitch (dict): Dictionary mapping indices to pitch values.
        output_file (str): Desired filename for the output MIDI file.
        step_duration (float): Duration of each time step in beats.
        velocity (int): Velocity (volume) of the notes.

    Returns:
        str: Path to the saved MIDI file.
    """
    print(f"Converting the generated sequence into MIDI format as '{output_file}'...")

    # Initialize a PrettyMIDI object
    midi_obj = pretty_midi.PrettyMIDI()

    # Create an Instrument instance (e.g., Acoustic Grand Piano)
    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)

    current_time = 0.0
    active_notes = {}

    for pitches in generated_sequence:
        # Determine pitches to deactivate
        pitches_to_remove = set(active_notes.keys()) - set(pitches)
        for pitch in pitches_to_remove:
            note = active_notes[pitch]
            note.end = current_time
            piano.notes.append(note)
            del active_notes[pitch]

        # Activate new pitches or extend existing ones
        for pitch in pitches:
            if pitch not in active_notes:
                # Start a new note
                new_note = pretty_midi.Note(
                    velocity=velocity,
                    pitch=pitch,
                    start=current_time,
                    end=current_time + step_duration
                )
                active_notes[pitch] = new_note
            else:
                # Extend the duration of an already active note
                active_notes[pitch].end = current_time + step_duration

        # Advance the current time
        current_time += step_duration

    # Turn off any remaining active notes at the end
    for pitch, note in active_notes.items():
        note.end = current_time
        piano.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    midi_obj.instruments.append(piano)

    # Save the MIDI file
    midi_obj.write(output_file)
    print(f"MIDI file '{output_file}' has been successfully saved.\n")

    return output_file

# 5. Function to Execute the Complete Music Generation Workflow

def perform_music_generation(model_path='final_lstm_music_model.keras',
                             pitch_to_idx_path='pitch_to_index_lstm.pkl',
                             idx_to_pitch_path='index_to_pitch_lstm.pkl',
                             seeds_file='lstm_train_seeds.pkl',
                             output_midi='lstm_generated_music.mid',
                             seq_length=64,
                             generation_steps=200,
                             temperature=1.0,
                             activation_threshold=0.4,
                             step_duration=0.3,
                             note_velocity=80):
    """
    Orchestrates the entire process of generating new music using the trained LSTM model.

    Args:
        model_path (str): Path to the trained LSTM model file.
        pitch_to_idx_path (str): Path to the pitch-to-index mapping file.
        idx_to_pitch_path (str): Path to the index-to-pitch mapping file.
        seeds_file (str): Path to the serialized training sequences file.
        output_midi (str): Desired name for the output MIDI file.
        seq_length (int): Number of previous time steps to consider.
        generation_steps (int): Total number of new time steps to generate.
        temperature (float): Controls the randomness of predictions.
        activation_threshold (float): Threshold to determine active pitches.
        step_duration (float): Duration of each time step in beats.
        note_velocity (int): Velocity (volume) of the notes.

    Returns:
        None
    """
    # Load the trained LSTM model and pitch mappings
    lstm_model, pitch_to_idx, idx_to_pitch = load_trained_model(model_path, pitch_to_idx_path, idx_to_pitch_path)

    # Load training seeds for initializing the generation process
    training_seeds = load_seed_sequences(seeds_file)

    # Generate a new sequence of pitches using the LSTM model
    new_music_sequence = generate_music_sequence(
        model=lstm_model,
        seeds=training_seeds,
        idx_to_pitch=idx_to_pitch,
        seq_length=seq_length,
        generation_steps=generation_steps,
        temperature=temperature,
        activation_threshold=activation_threshold
    )

    # Convert the generated sequence into a MIDI file
    midi_file_path = convert_sequence_to_midi(
        generated_sequence=new_music_sequence,
        idx_to_pitch=idx_to_pitch,
        output_file=output_midi,
        step_duration=step_duration,
        velocity=note_velocity
    )

    print(f"Music generation complete. MIDI file saved at: {midi_file_path}")

# Run the music generation process
if __name__ == "__main__":
    perform_music_generation(
        model_path='final_lstm_music_model.keras',
        pitch_to_idx_path='pitch_to_index_lstm.pkl',
        idx_to_pitch_path='index_to_pitch_lstm.pkl',
        seeds_file='lstm_train_seeds.pkl',
        output_midi='lstm_generated_music.mid',
        seq_length=64,
        generation_steps=200,       # Adjust the number of steps as needed
        temperature=1.0,            # Modify to control randomness
        activation_threshold=0.4,   # Tune to balance pitch activation
        step_duration=0.3,          # Set the duration per time step in beats
        note_velocity=80            # Define the volume for generated notes
    )
